# Milestone 1

### Data

In [1]:
import numpy as np
X = np.array([[0,0],[0,1],[1,0],[1,1]], dtype=np.float32)
y = np.array([[0],[1],[1],[0]], dtype=np.float32)

### Build network with your library

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from lib import Sequential, Dense, Tanh, Sigmoid, MSELoss, SGD

model = Sequential([
    Dense(2,4), Tanh(),
    Dense(4,1), Sigmoid()
])
opt = SGD(lr=1.0)

# rescale weights for XOR only
for lyr in model.layers:
    if hasattr(lyr, 'W'):
        lyr.W = np.random.randn(*lyr.W.shape) * 1.0

### Training loop

In [5]:
losses=[]
for epoch in range(5000):
    loss = model.train_step(X, y)
    opt.step(model.layers)
    losses.append(loss)
    if epoch % 500 == 0:
        print(epoch, loss)

0 0.25219693696914786
500 0.0015543438268264748
1000 0.0006825703486202151
1500 0.0004312597234859038
2000 0.0003135387063069906
2500 0.0002456392857838166
3000 0.00020158921515686158
3500 0.00017075651102995632
4000 0.00014799609165193833
4500 0.00013051988388224103


### Assert correctness

In [6]:
pred = model.forward(X)
print("Raw preds:\n", pred)
print("Rounded:\n", np.round(pred))

Raw preds:
 [[0.00490199]
 [0.98731338]
 [0.98912596]
 [0.01278777]]
Rounded:
 [[0.]
 [1.]
 [1.]
 [0.]]


### Gradient-check block

In [7]:
def grad_check(net, X, y, eps=1e-5):
    loss_fn = MSELoss()
    numeric = []
    analytic = net.layers[0].dW.copy()  # shape of first W

    # numeric
    for idx in np.ndindex(analytic.shape):
        net.layers[0].W[idx] += eps
        plus = loss_fn(y, net.forward(X))
        net.layers[0].W[idx] -= 2*eps
        minus = loss_fn(y, net.forward(X))
        net.layers[0].W[idx] += eps
        numeric.append((plus - minus)/(2*eps))
    numeric = np.array(numeric).reshape(analytic.shape)

    # analytic (already stored by net.train_step)
    net.train_step(X, y)  # refreshes dW
    analytic = net.layers[0].dW

    return np.abs(analytic - numeric).max()

print('Max diff:', grad_check(model, X, y))  # <1e-7 → pass

Max diff: 5.4243430900420206e-14
